In [11]:
import os
from dotenv import load_dotenv
import openai
import autogen
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import pyodbc
from mysql.connector import errorcode

load_dotenv()

True

In [12]:
query_maker_gpt_system_prompt = '''You are MySQL Query Generator. Kindly generate the sql query only and use only the listed columns in 
below schema. Do not use any column by yourself. 

Below is the Schema of the available tables to make the sql queries. Create and return only one query.

CREATE TABLE `Inventory` (
    ProductID int NOT NULL,
    ProductName varchar DEFAULT NULL,
    Quantity int DEFAULT NULL,
    UnitPrice int DEFAULT NULL
);

CREATE TABLE `Sales` (
    ProductID int NOT NULL,
    Quantity int DEFAULT NULL,
    SaleDate date DEFAULT NULL
);

The [dbo].[Inventory] table contains all sold products information and the [dbo].[Sales] table all dates of sold items.

User Input: 
'''

admin_prompt = "Admin"
data_engineer_prompt = '''Do not change user input. You have the opportunity to advise the Admin on selecting the appropriate function, along with the required arguments. The "query_maker" function is designed to accept human input as an argument and construct the SQL query. Meanwhile, the "run_sql_query" function is responsible for executing the query. Please refrain from independently crafting SQL queries.
Once you receive the results from the Admin in response to the SQL query, ensure that you interpret them accurately. You are also authorized to create SQL queries tailored to user input. Subsequently, execute the query and provide the results. In the event of any errors, please rectify them and rerun the query, and then present the answer.
If the sql query result is empty, then just say we do not have this mobile in our stock.
'''

In [13]:
api_key=os.getenv('OPENAI_API_KEY')

# Set your LLms Endpoint
config_list_gpt_turbo = autogen.config_list_from_models(model_list=[ "gpt-4"])

def query_maker(user_input):
    # make sql queries using LLM chain
    openaiLLM = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7,
                                        openai_api_key=api_key, cache=False)
    prompt_template = PromptTemplate.from_template(
        "{system_prompt} + '\n' +  {user_input}.")

    chain = LLMChain(llm=openaiLLM, prompt=prompt_template)
    query=chain.run({"system_prompt": query_maker_gpt_system_prompt, "user_input": user_input})
    return query

def run_sql_query(sql_query):
    
    connectionString = f'DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={os.getenv('azure_sql_host')};DATABASE={os.getenv('database')};UID={os.getenv('user')};PWD={os.getenv('password')}'

    conn = pyodbc.connect(connectionString)

    cursor = conn.cursor()
    try:
        # Execute SQL queries
        cursor.execute(sql_query)
        result = cursor.fetchall()
    except Exception as e:
        return e

    return result

gpt_turbo_simple_config = {
    "temperature": 0.5,
    "config_list": config_list_gpt_turbo}


gpt_turbo_config = {
    "temperature": 0.7,
    "config_list": config_list_gpt_turbo,
    "functions" : [
    {
        "name": "query_maker",
        "description": "generates sql query as per user input",
        "parameters": {
            "type": "object",
            "properties": {
                "user_input": {
                    "type": "string",
                    "description": "This is the input from the user side.",
                }
                ,
            },
            "required": ["user_input"],
        },
    },

{
        "name": "run_sql_query",
        "description": "This function is used to run sql query against user input to get the results.",
        "parameters": {
            "type": "object",
            "properties": {
                "sql_query": {
                    "type": "string",
                    "description": "This is the mysql query.",
                }
                ,
            },
            "required": ["sql_query"],
        },
    }



    ]
}
function_map={"query_maker": query_maker ,"run_sql_query": run_sql_query}
termination_msg="If everything looks good, respond with Approved."

def is_termination_msg(content):
    have_content=content.get("content", None) is not None
    if have_content and "Approved" in content["content"]:
        return True
    else:
        return False

In [ ]:
user_proxy = autogen.UserProxyAgent(
   name="Admin",
   system_message= admin_prompt + termination_msg,
   human_input_mode="NEVER",
    is_termination_msg=is_termination_msg
)

engineer = autogen.AssistantAgent(
    name="Data_Engineer",
    llm_config=gpt_turbo_config,
    system_message=data_engineer_prompt + termination_msg,
    function_map=function_map
)

# register the functions
user_proxy.register_function(function_map={"query_maker": query_maker ,"run_sql_query": run_sql_query},)

user_proxy.initiate_chat( engineer,
    message="""Which product have we sold the most of in November?""", clear_history=True
)

user_proxy_chat=user_proxy.chat_messages
engineer_chat=engineer.chat_messages

Admin (to Data_Engineer):

Which product have we sold the most of in November?

--------------------------------------------------------------------------------
Data_Engineer (to Admin):

***** Suggested function call: query_maker *****
Arguments: 
{
  "user_input": "Which product have we sold the most of in November?"
}
************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION query_maker...
Admin (to Data_Engineer):

***** Response from calling function (query_maker) *****
SELECT Inventory.ProductName
FROM Inventory
INNER JOIN Sales ON Inventory.ProductID = Sales.ProductID
WHERE MONTH(Sales.SaleDate) = 11
GROUP BY Inventory.ProductID
ORDER BY SUM(Sales.Quantity) DESC
LIMIT 1;
********************************************************

--------------------------------------------------------------------------------
Data_Engineer (to Admin):

***** Suggested function call: run_sql_query 